# SW-9-SHACL
**Navigation** : [<< 8-PythonRDF](SW-8-PythonRDF.ipynb) | [Index](README.md) | [10-JSONLD >>](SW-10-JSONLD.ipynb)

## SHACL : Validation et Qualite des Donnees RDF
Ce notebook explore SHACL (Shapes Constraint Language), la recommandation W3C pour la validation de donnees RDF. Contrairement a OWL qui raisonne sous l'hypothese du monde ouvert, SHACL permet de definir des contraintes concretes et de verifier que les donnees d'un graphe RDF les respectent.

### Objectifs d'apprentissage
A la fin de ce notebook, vous saurez :
1. Definir des shapes SHACL pour contraindre un graphe RDF
2. Valider des donnees avec pySHACL et interpreter le rapport
3. Creer des shapes custom programmatiquement
4. Utiliser SHACL pour garantir la qualite des donnees

### Prerequis
- Notebook SW-8-PythonRDF complete
### Duree estimee : 45 minutes

---

## 0. Installation des dependances

Installons les deux bibliotheques necessaires :
- **rdflib** : manipulation de graphes RDF (deja utilisee dans SW-8)
- **pyshacl** : moteur de validation SHACL pour Python

In [1]:
%pip install -q rdflib pyshacl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Verifions que les installations se sont bien passees en important les modules et en affichant leurs versions.

In [2]:
import rdflib
import pyshacl

print(f"rdflib   : {rdflib.__version__}")
print(f"pyshacl  : {pyshacl.__version__}")
print("Installation OK.")

rdflib   : 7.6.0
pyshacl  : 0.31.0
Installation OK.


---

## 1. Introduction a SHACL

### Le probleme : l'hypothese du monde ouvert

En RDF et OWL, on travaille sous l'**hypothese du monde ouvert** (*Open World Assumption*, OWA) : l'absence d'information ne signifie pas que cette information est fausse. Si un graphe RDF ne mentionne pas l'age d'une personne, OWL considere simplement que l'age est *inconnu*, pas qu'il est absent.

C'est un probleme pour la **qualite des donnees**. Dans une application reelle, on veut pouvoir dire :
- "Chaque personne **doit** avoir un nom"
- "L'age **doit** etre un entier positif"
- "Un email **doit** respecter un format precis"

OWL ne peut pas exprimer ces contraintes de validation. C'est la raison d'etre de **SHACL** (*Shapes Constraint Language*).

### SHACL : Shapes Constraint Language

SHACL est une **recommandation W3C** publiee en juillet 2017. Elle definit un vocabulaire RDF pour decrire des contraintes ("shapes") que les donnees RDF doivent respecter.

| Aspect | OWL (monde ouvert) | SHACL (monde ferme) |
|--------|-------------------|---------------------|
| **Hypothese** | Monde ouvert (OWA) | Monde ferme (CWA) |
| **Objectif** | Inference, raisonnement | Validation, qualite |
| **Absence de donnee** | Inconnue | Violation potentielle |
| **`minCount 1`** | Implique l'existence | Exige la presence |
| **Analogie** | Definition d'un concept | Formulaire de saisie |
| **Standard W3C** | OWL 2 (2012) | SHACL 1.0 (2017) |

> **Point cle** : OWL decrit *ce qui est vrai* dans un domaine. SHACL decrit *ce que les donnees doivent respecter*. Les deux sont complementaires : OWL pour modeliser, SHACL pour valider.

---

## 2. Concepts fondamentaux de SHACL

SHACL repose sur deux types de formes (*shapes*) :

### NodeShape et PropertyShape

| Concept | URI SHACL | Description |
|---------|-----------|-------------|
| **NodeShape** | `sh:NodeShape` | Forme qui s'applique a un noeud RDF (une ressource) |
| **PropertyShape** | `sh:PropertyShape` | Forme qui contraint une propriete specifique |
| **targetClass** | `sh:targetClass` | Cible tous les noeuds d'une classe donnee |
| **targetNode** | `sh:targetNode` | Cible un noeud specifique par son URI |
| **property** | `sh:property` | Lie une NodeShape a ses PropertyShapes |
| **path** | `sh:path` | Designe la propriete RDF a contraindre |

### Architecture d'une shape SHACL

```turtle
ex:PersonShape a sh:NodeShape ;       # Forme pour les personnes
    sh:targetClass foaf:Person ;       # Cible : toutes les foaf:Person
    sh:property [                      # Contrainte sur une propriete
        sh:path foaf:name ;            # Propriete ciblee
        sh:minCount 1 ;               # Au moins une valeur
        sh:datatype xsd:string ;       # Type attendu
    ] .
```

### Principaux types de contraintes

| Categorie | Contrainte | Propriete SHACL | Exemple |
|-----------|------------|-----------------|---------|  
| **Cardinalite** | Minimum / Maximum | `sh:minCount`, `sh:maxCount` | `sh:minCount 1` |
| **Type** | Type de donnee | `sh:datatype` | `sh:datatype xsd:string` |
| **Classe** | Classe RDF | `sh:class` | `sh:class foaf:Person` |
| **Chaine** | Longueur minimale | `sh:minLength` | `sh:minLength 2` |
| **Chaine** | Motif regex | `sh:pattern` | `sh:pattern "^[A-Z]"` |
| **Numerique** | Borne inclusive | `sh:minInclusive`, `sh:maxInclusive` | `sh:minInclusive 0` |

---

## 3. Explorer une shape SHACL existante

Le fichier `data/person-shape.ttl` contient une forme SHACL pour valider des instances de `foaf:Person`. Chargeons-le et examinons les contraintes definies.

In [3]:
from rdflib import Graph, Namespace, URIRef, Literal
from rdflib.namespace import RDF, RDFS, XSD, FOAF

# Namespaces
SH = Namespace("http://www.w3.org/ns/shacl#")
EX = Namespace("http://example.org/")

# Charger les formes SHACL
shapes_graph = Graph()
shapes_graph.parse("data/person-shape.ttl", format="turtle")

print(f"Graphe des formes charge : {len(shapes_graph)} triples")
print()
print("=" * 60)
print("Contenu de person-shape.ttl :")
print("=" * 60)
print(shapes_graph.serialize(format="turtle"))

Graphe des formes charge : 25 triples

Contenu de person-shape.ttl :
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:PersonShape a sh:NodeShape ;
    sh:property [ sh:datatype xsd:string ;
            sh:maxCount 1 ;
            sh:message "Une personne doit avoir exactement un nom (string, min 2 caracteres)"@fr ;
            sh:minCount 1 ;
            sh:minLength 2 ;
            sh:path foaf:name ],
        [ sh:maxCount 1 ;
            sh:message "L'email doit etre au format mailto:user@domain.tld"@fr ;
            sh:path foaf:mbox ;
            sh:pattern "^mailto:.+@.+\\..+$" ],
        [ sh:datatype xsd:integer ;
            sh:maxCount 1 ;
            sh:maxInclusive 150 ;
            sh:message "L'age doit etre un entier entre 0 et 150"@fr ;
            sh:minInclusive 0 ;
            sh:path foaf:age ],
        [ sh:class foaf:Person ;
         

### Analyse structurelle par programmation

Plutot que de lire le fichier Turtle a l'oeil, parcourons le graphe SHACL avec rdflib pour identifier automatiquement les NodeShapes, leurs cibles et les contraintes de chaque propriete.

In [4]:
# Parcourir les NodeShapes et leurs contraintes
for shape in shapes_graph.subjects(RDF.type, SH.NodeShape):
    print(f"NodeShape : {shape}")
    
    # Cible
    for target in shapes_graph.objects(shape, SH.targetClass):
        print(f"  Cible (targetClass) : {target}")
    
    # Proprietes contraintes
    print(f"  Contraintes de propriete :")
    for prop_shape in shapes_graph.objects(shape, SH.property):
        path = list(shapes_graph.objects(prop_shape, SH.path))
        path_str = str(path[0]) if path else "?"
        
        # Collecter les contraintes SHACL (hors path et message)
        constraints = []
        for pred, obj in shapes_graph.predicate_objects(prop_shape):
            pred_local = str(pred).split("#")[-1] if "#" in str(pred) else str(pred).split("/")[-1]
            if pred_local not in ("path", "message") and str(pred).startswith(str(SH)):
                constraints.append(f"{pred_local}={obj}")
        
        # Message
        messages = list(shapes_graph.objects(prop_shape, SH.message))
        msg = str(messages[0]) if messages else "-"
        
        print(f"    - {path_str.split('/')[-1]}")
        for c in constraints:
            print(f"        {c}")
        print(f"        Message: {msg}")

NodeShape : http://example.org/PersonShape
  Cible (targetClass) : http://xmlns.com/foaf/0.1/Person
  Contraintes de propriete :
    - name
        minCount=1
        maxCount=1
        datatype=http://www.w3.org/2001/XMLSchema#string
        minLength=2
        Message: Une personne doit avoir exactement un nom (string, min 2 caracteres)
    - mbox
        maxCount=1
        pattern=^mailto:.+@.+\..+$
        Message: L'email doit etre au format mailto:user@domain.tld
    - age
        maxCount=1
        datatype=http://www.w3.org/2001/XMLSchema#integer
        minInclusive=0
        maxInclusive=150
        Message: L'age doit etre un entier entre 0 et 150
    - knows
        class=http://xmlns.com/foaf/0.1/Person
        Message: La propriete knows doit pointer vers une autre Person


### Interpretation : Structure de la forme PersonShape

La forme `ex:PersonShape` cible toutes les instances de `foaf:Person` et impose 4 contraintes :

| Propriete | Contraintes | Signification |
|-----------|-------------|---------------|
| `foaf:name` | minCount=1, maxCount=1, datatype=string, minLength=2 | Exactement un nom, chaine d'au moins 2 caracteres |
| `foaf:mbox` | maxCount=1, pattern=`^mailto:.+@.+\\..+$` | Au plus un email, format mailto: valide |
| `foaf:age` | maxCount=1, datatype=integer, minInclusive=0, maxInclusive=150 | Au plus un age, entier entre 0 et 150 |
| `foaf:knows` | class=foaf:Person | Les personnes connues doivent etre des foaf:Person |

> **Note** : `foaf:mbox` et `foaf:age` n'ont pas de `minCount`, ils sont donc optionnels. Seul `foaf:name` est obligatoire (minCount=1).

---

## 4. pySHACL : Validation en Python

### La fonction `validate()`

[pySHACL](https://github.com/RDFLib/pySHACL) est l'implementation Python de reference pour la validation SHACL. Sa fonction principale `validate()` accepte les parametres suivants :

| Parametre | Type | Description |
|-----------|------|-------------|
| `data_graph` | Graph / str | Le graphe de donnees a valider (1er argument positionnel) |
| `shacl_graph` | Graph / str | Le graphe de formes SHACL |
| `inference` | str | Mode d'inference : `'none'`, `'rdfs'`, `'owlrl'`, `'both'` |
| `abort_on_first` | bool | Arreter a la premiere violation (defaut: False) |
| `advanced` | bool | Activer SHACL Advanced Features (regles, etc.) |
| `inplace` | bool | Modifier le graphe de donnees en place (pour les regles) |

La fonction retourne un triplet `(conforms, results_graph, results_text)` :
- `conforms` : booleen, True si toutes les contraintes sont satisfaites
- `results_graph` : graphe RDF contenant les resultats detailles
- `results_text` : rapport texte lisible

### Charger les donnees de test

Le fichier `data/person-data.ttl` contient 7 personnes dont **5 presentent des erreurs volontaires**. Chargeons-le et examinons son contenu.

In [5]:
# Charger les donnees de test
data_graph = Graph()
data_graph.parse("data/person-data.ttl", format="turtle")

print(f"Graphe de donnees charge : {len(data_graph)} triples")
print()

# Lister les personnes avec leurs proprietes
print("Personnes dans le graphe :")
print(f"{'Personne':<12s} {'Nom':<20s} {'Age':<8s} {'Email':<25s} {'Knows':<12s}")
print("-" * 77)

for person in sorted(data_graph.subjects(RDF.type, FOAF.Person), key=str):
    name = list(data_graph.objects(person, FOAF.name))
    age = list(data_graph.objects(person, FOAF.age))
    mbox = list(data_graph.objects(person, FOAF.mbox))
    knows = list(data_graph.objects(person, FOAF.knows))
    
    name_str = str(name[0]) if name else "[ABSENT]"
    age_str = str(age[0]) if age else "-"
    mbox_str = str(mbox[0]).split('/')[-1] if mbox else "-"
    knows_str = str(knows[0]).split('/')[-1] if knows else "-"
    
    print(f"{str(person).split('/')[-1]:<12s} {name_str:<20s} {age_str:<8s} {mbox_str:<25s} {knows_str:<12s}")

Graphe de donnees charge : 25 triples

Personnes dans le graphe :
Personne     Nom                  Age      Email                     Knows       
-----------------------------------------------------------------------------
alice        Alice Dupont         30       mailto:alice@example.com  bob         
bob          Bob Martin           25       -                         alice       
charlie      [ABSENT]             40       -                         -           
diana        Diana Leroy          -5       -                         -           
eve          Eve Bernard          -        not-an-email              -           
frank        F                    55       -                         -           
grace        Grace Moreau         -        -                         myDog       


### Execution de la validation

Validons les donnees contre les formes SHACL. L'option `inference='rdfs'` permet a pySHACL de prendre en compte les inferences RDFS (sous-classes, sous-proprietes) lors de la validation.

In [6]:
from pyshacl import validate

# Validation SHACL
conforms, results_graph, results_text = validate(
    data_graph,
    shacl_graph=shapes_graph,
    inference='rdfs',
    abort_on_first=False
)

print(f"Les donnees sont conformes : {conforms}")
print()
print("=" * 60)
print("Rapport de validation :")
print("=" * 60)
print(results_text)

Les donnees sont conformes : False

Rapport de validation :
Validation Report
Conforms: False
Results (5):
Constraint Violation in ClassConstraintComponent (http://www.w3.org/ns/shacl#ClassConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class foaf:Person ; sh:message Literal("La propriete knows doit pointer vers une autre Person", lang=fr) ; sh:path foaf:knows ]
	Focus Node: ex:grace
	Value Node: ex:myDog
	Result Path: foaf:knows
	Message: La propriete knows doit pointer vers une autre Person
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:string ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:message Literal("Une personne doit avoir exactement un nom (string, min 2 caracteres)", lang=fr) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:minLength Literal("2", datatype=xsd:integer) ; sh:path foaf:name ]
	Focus Node: ex:charlie
	Result Path: 

### Analyse structuree du graphe de resultats

Le rapport texte est utile pour un humain, mais on peut aussi **analyser le graphe de resultats** par programmation. Chaque violation est un noeud de type `sh:ValidationResult` avec des proprietes structurees.

In [7]:
# Analyser le graphe de resultats par programmation
violations = []

for result in results_graph.subjects(RDF.type, SH.ValidationResult):
    focus = list(results_graph.objects(result, SH.focusNode))
    path = list(results_graph.objects(result, SH.resultPath))
    severity = list(results_graph.objects(result, SH.resultSeverity))
    message = list(results_graph.objects(result, SH.resultMessage))
    value = list(results_graph.objects(result, SH.value))
    
    violations.append({
        "focus": str(focus[0]).split("/")[-1] if focus else "?",
        "path": str(path[0]).split("/")[-1] if path else "-",
        "severity": str(severity[0]).split("#")[-1] if severity else "?",
        "message": str(message[0]) if message else "-",
        "value": str(value[0]) if value else "-"
    })

# Affichage tabulaire
print(f"Nombre total de violations : {len(violations)}")
print()
print(f"{'Noeud':<12} {'Propriete':<10} {'Severite':<12} {'Message'}")
print("-" * 90)
for v in sorted(violations, key=lambda x: x["focus"]):
    print(f"{v['focus']:<12} {v['path']:<10} {v['severity']:<12} {v['message'][:55]}")

Nombre total de violations : 5

Noeud        Propriete  Severite     Message
------------------------------------------------------------------------------------------
charlie      name       Violation    Une personne doit avoir exactement un nom (string, min 
diana        age        Violation    L'age doit etre un entier entre 0 et 150
eve          mbox       Violation    L'email doit etre au format mailto:user@domain.tld
frank        name       Violation    Une personne doit avoir exactement un nom (string, min 
grace        knows      Violation    La propriete knows doit pointer vers une autre Person


### Interpretation : Les 5 erreurs detectees

Le fichier `person-data.ttl` contenait 5 erreurs volontaires, toutes detectees par pySHACL :

| Personne | Propriete | Type de violation | Explication |
|----------|-----------|-------------------|-------------|
| **alice** | - | Aucune | Conforme : toutes les proprietes respectent les contraintes |
| **bob** | - | Aucune | Conforme (email optionnel, pas de minCount sur mbox) |
| **charlie** | `foaf:name` | `minCount` | Pas de nom alors que `minCount=1` est exige |
| **diana** | `foaf:age` | `minInclusive` | Age negatif (-5) alors que `minInclusive=0` |
| **eve** | `foaf:mbox` | `pattern` | Email `not-an-email` ne correspond pas au motif `^mailto:` |
| **frank** | `foaf:name` | `minLength` | Nom "F" (1 caractere) trop court, `minLength=2` |
| **grace** | `foaf:knows` | `class` | `ex:myDog` est un `ex:Dog`, pas un `foaf:Person` |

**Points cles** :
1. Alice et Bob passent la validation car leurs donnees sont completes et conformes
2. Bob n'a pas d'email mais c'est conforme : `foaf:mbox` n'a pas de `minCount`
3. Chaque violation est independante : un meme noeud pourrait cumuler plusieurs violations

---

## 5. Ecrire des shapes custom en Python

Plutot que de charger des fichiers Turtle, on peut construire des formes SHACL **directement en Python** avec rdflib. Cela permet de creer des validations dynamiques, generees par du code.

### Catalogue des contraintes

| Categorie | Contrainte | Propriete SHACL | Description |
|-----------|------------|-----------------|-------------|
| Cardinalite | Minimum | `sh:minCount` | Nombre minimum de valeurs |
| Cardinalite | Maximum | `sh:maxCount` | Nombre maximum de valeurs |
| Type | Type de donnee | `sh:datatype` | Type XSD attendu |
| Type | Classe RDF | `sh:class` | La valeur doit etre d'une classe donnee |
| Type | Type de noeud | `sh:nodeKind` | IRI, BlankNode ou Literal |
| Chaine | Motif regex | `sh:pattern` | Expression reguliere |
| Chaine | Longueur min | `sh:minLength` | Nombre minimum de caracteres |
| Chaine | Longueur max | `sh:maxLength` | Nombre maximum de caracteres |
| Numerique | Borne min | `sh:minInclusive` | Valeur minimale (inclusive) |
| Numerique | Borne max | `sh:maxInclusive` | Valeur maximale (inclusive) |
| Valeur | Valeur exacte | `sh:hasValue` | Doit contenir cette valeur |
| Valeur | Liste | `sh:in` | Doit etre dans cette liste |
| Unicite | Langue unique | `sh:uniqueLang` | Une seule valeur par tag de langue |

### Construire une ArticleShape

Creons une forme SHACL pour valider des articles (`schema:Article`) avec les contraintes suivantes :
- Titre obligatoire (string, min 5 caracteres)
- Au moins un auteur (doit etre un `schema:Person`)
- Date de publication optionnelle (format `xsd:date`)
- Nombre de mots optionnel (entier entre 100 et 50000)

In [8]:
from rdflib import Graph, Namespace, URIRef, Literal, BNode
from rdflib.namespace import RDF, XSD, FOAF

SH = Namespace("http://www.w3.org/ns/shacl#")
EX = Namespace("http://example.org/")
SCHEMA = Namespace("http://schema.org/")

# Creer un nouveau graphe de formes
custom_shapes = Graph()
custom_shapes.bind("sh", SH)
custom_shapes.bind("ex", EX)
custom_shapes.bind("schema", SCHEMA)
custom_shapes.bind("xsd", XSD)

# --- NodeShape pour un Article ---
article_shape = EX.ArticleShape
custom_shapes.add((article_shape, RDF.type, SH.NodeShape))
custom_shapes.add((article_shape, SH.targetClass, SCHEMA.Article))

# Contrainte 1 : titre obligatoire (minCount + datatype + minLength)
title_prop = BNode()
custom_shapes.add((article_shape, SH.property, title_prop))
custom_shapes.add((title_prop, SH.path, SCHEMA.headline))
custom_shapes.add((title_prop, SH.minCount, Literal(1)))
custom_shapes.add((title_prop, SH.maxCount, Literal(1)))
custom_shapes.add((title_prop, SH.datatype, XSD.string))
custom_shapes.add((title_prop, SH.minLength, Literal(5)))
custom_shapes.add((title_prop, SH.message, Literal("Un article doit avoir un titre (string, min 5 caracteres)", lang="fr")))

# Contrainte 2 : auteur obligatoire, doit etre un schema:Person
author_prop = BNode()
custom_shapes.add((article_shape, SH.property, author_prop))
custom_shapes.add((author_prop, SH.path, SCHEMA.author))
custom_shapes.add((author_prop, SH.minCount, Literal(1)))
custom_shapes.add((author_prop, SH["class"], SCHEMA.Person))
custom_shapes.add((author_prop, SH.message, Literal("Un article doit avoir au moins un auteur (schema:Person)", lang="fr")))

# Contrainte 3 : date de publication optionnelle (xsd:date)
date_prop = BNode()
custom_shapes.add((article_shape, SH.property, date_prop))
custom_shapes.add((date_prop, SH.path, SCHEMA.datePublished))
custom_shapes.add((date_prop, SH.maxCount, Literal(1)))
custom_shapes.add((date_prop, SH.datatype, XSD.date))
custom_shapes.add((date_prop, SH.message, Literal("La date de publication doit etre au format xsd:date", lang="fr")))

# Contrainte 4 : nombre de mots optionnel (entier entre 100 et 50000)
words_prop = BNode()
custom_shapes.add((article_shape, SH.property, words_prop))
custom_shapes.add((words_prop, SH.path, SCHEMA.wordCount))
custom_shapes.add((words_prop, SH.maxCount, Literal(1)))
custom_shapes.add((words_prop, SH.datatype, XSD.integer))
custom_shapes.add((words_prop, SH.minInclusive, Literal(100)))
custom_shapes.add((words_prop, SH.maxInclusive, Literal(50000)))
custom_shapes.add((words_prop, SH.message, Literal("Le nombre de mots doit etre entre 100 et 50000", lang="fr")))

print("Forme ArticleShape creee avec 4 contraintes de propriete")
print()
print(custom_shapes.serialize(format="turtle"))

Forme ArticleShape creee avec 4 contraintes de propriete

@prefix ex: <http://example.org/> .
@prefix schema1: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:ArticleShape a sh:NodeShape ;
    sh:property [ sh:class schema1:Person ;
            sh:message "Un article doit avoir au moins un auteur (schema:Person)"@fr ;
            sh:minCount 1 ;
            sh:path schema1:author ],
        [ sh:datatype xsd:integer ;
            sh:maxCount 1 ;
            sh:maxInclusive 50000 ;
            sh:message "Le nombre de mots doit etre entre 100 et 50000"@fr ;
            sh:minInclusive 100 ;
            sh:path schema1:wordCount ],
        [ sh:datatype xsd:string ;
            sh:maxCount 1 ;
            sh:message "Un article doit avoir un titre (string, min 5 caracteres)"@fr ;
            sh:minCount 1 ;
            sh:minLength 5 ;
            sh:path schema1:headline ],
        [ sh:datatype xsd:date ;
       

### Test de la forme personnalisee

Creons un jeu de donnees avec des articles valides et invalides pour verifier notre forme ArticleShape.

In [9]:
# Creer des donnees de test pour ArticleShape
test_data = Graph()
test_data.bind("ex", EX)
test_data.bind("schema", SCHEMA)

# Auteur valide
test_data.add((EX.jean, RDF.type, SCHEMA.Person))
test_data.add((EX.jean, SCHEMA.name, Literal("Jean Dupont")))

# Article 1 : valide (toutes les proprietes correctes)
test_data.add((EX.article1, RDF.type, SCHEMA.Article))
test_data.add((EX.article1, SCHEMA.headline, Literal("Introduction au Web Semantique", datatype=XSD.string)))
test_data.add((EX.article1, SCHEMA.author, EX.jean))
test_data.add((EX.article1, SCHEMA.datePublished, Literal("2024-01-15", datatype=XSD.date)))
test_data.add((EX.article1, SCHEMA.wordCount, Literal(5000, datatype=XSD.integer)))

# Article 2 : titre trop court ("RDF" = 3 caracteres < 5)
test_data.add((EX.article2, RDF.type, SCHEMA.Article))
test_data.add((EX.article2, SCHEMA.headline, Literal("RDF", datatype=XSD.string)))
test_data.add((EX.article2, SCHEMA.author, EX.jean))

# Article 3 : pas de titre, pas d'auteur
test_data.add((EX.article3, RDF.type, SCHEMA.Article))
test_data.add((EX.article3, SCHEMA.wordCount, Literal(200, datatype=XSD.integer)))

# Article 4 : nombre de mots hors bornes (10 < 100)
test_data.add((EX.article4, RDF.type, SCHEMA.Article))
test_data.add((EX.article4, SCHEMA.headline, Literal("Article avec trop peu de mots", datatype=XSD.string)))
test_data.add((EX.article4, SCHEMA.author, EX.jean))
test_data.add((EX.article4, SCHEMA.wordCount, Literal(10, datatype=XSD.integer)))

# Valider
conforms2, results2, text2 = validate(
    test_data,
    shacl_graph=custom_shapes,
    inference='rdfs',
    abort_on_first=False
)

print(f"Conforme : {conforms2}")
print()
print(text2)

Conforme : False

Validation Report
Conforms: False
Results (4):
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:class schema1:Person ; sh:message Literal("Un article doit avoir au moins un auteur (schema:Person)", lang=fr) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:path schema1:author ]
	Focus Node: ex:article3
	Result Path: schema1:author
	Message: Un article doit avoir au moins un auteur (schema:Person)
Constraint Violation in MinCountConstraintComponent (http://www.w3.org/ns/shacl#MinCountConstraintComponent):
	Severity: sh:Violation
	Source Shape: [ sh:datatype xsd:string ; sh:maxCount Literal("1", datatype=xsd:integer) ; sh:message Literal("Un article doit avoir un titre (string, min 5 caracteres)", lang=fr) ; sh:minCount Literal("1", datatype=xsd:integer) ; sh:minLength Literal("5", datatype=xsd:integer) ; sh:path schema1:headline ]
	Focus Node: ex:article3
	Resul

### Interpretation : Validation des articles

| Article | Violations | Detail |
|---------|-----------|--------|
| **article1** | 0 | Conforme : titre, auteur, date et wordCount respectent toutes les contraintes |
| **article2** | 1 | Titre "RDF" trop court (3 < 5 caracteres requis par minLength) |
| **article3** | 2 | Pas de titre (minCount=1) + pas d'auteur (minCount=1) |
| **article4** | 1 | wordCount=10 < minInclusive=100 |

> **Bonne pratique** : Ajoutez toujours des messages explicites (`sh:message`) pour faciliter le diagnostic. Les messages en francais ou en anglais sont plus utiles que les messages generiques de pySHACL.

---

## 6. Severites SHACL

SHACL definit trois niveaux de severite pour les violations. Par defaut, toutes les contraintes ont la severite `sh:Violation`, mais on peut les ajuster.

| Severite | URI | Comportement | Usage typique |
|----------|-----|-------------|---------------|
| **Violation** | `sh:Violation` | Erreur bloquante (defaut) | Donnee manquante obligatoire |
| **Warning** | `sh:Warning` | Avertissement non bloquant | Donnee recommandee mais optionnelle |
| **Info** | `sh:Info` | Information, suggestion | Bonne pratique non imposee |

Le booleen `conforms` retourne `False` **uniquement** pour les `sh:Violation`. Les warnings et infos n'affectent pas la conformite.

In [10]:
# Demonstrer les 3 niveaux de severite
severity_shapes_ttl = """
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

ex:EmployeeShape a sh:NodeShape ;
    sh:targetClass ex:Employee ;
    # VIOLATION : le nom est obligatoire
    sh:property [
        sh:path foaf:name ;
        sh:minCount 1 ;
        sh:severity sh:Violation ;
        sh:message "Le nom est obligatoire (Violation)"@fr ;
    ] ;
    # WARNING : l'email est recommande mais pas obligatoire
    sh:property [
        sh:path foaf:mbox ;
        sh:minCount 1 ;
        sh:severity sh:Warning ;
        sh:message "L'email est recommande (Warning)"@fr ;
    ] ;
    # INFO : le telephone est souhaitable
    sh:property [
        sh:path ex:phone ;
        sh:minCount 1 ;
        sh:severity sh:Info ;
        sh:message "Le telephone est souhaitable (Info)"@fr ;
    ] .
"""

severity_shapes = Graph()
severity_shapes.parse(data=severity_shapes_ttl, format="turtle")

# Employe sans nom, sans email, sans telephone
severity_data = Graph()
severity_data.bind("ex", EX)
severity_data.add((EX.emp1, RDF.type, EX.Employee))
# Pas de nom, pas d'email, pas de telephone

conforms_sev, results_sev, text_sev = validate(
    severity_data,
    shacl_graph=severity_shapes,
    inference='rdfs',
    abort_on_first=False
)

print(f"Conforme : {conforms_sev}")
print()

# Classer les resultats par severite
for result in results_sev.subjects(RDF.type, SH.ValidationResult):
    severity = list(results_sev.objects(result, SH.resultSeverity))
    message = list(results_sev.objects(result, SH.resultMessage))
    sev_str = str(severity[0]).split("#")[-1] if severity else "?"
    msg_str = str(message[0]) if message else "-"
    print(f"  [{sev_str:<10s}] {msg_str}")

Conforme : False

  [Violation ] Le nom est obligatoire (Violation)
  [Warning   ] L'email est recommande (Warning)
  [Info      ] Le telephone est souhaitable (Info)


### Interpretation : Impact de la severite

Bien que les 3 contraintes soient violees, le booleen `conforms` ne depend que de `sh:Violation` :

| Severite | Contrainte violee | Impact sur `conforms` |
|----------|-------------------|----------------------|
| `sh:Violation` | Pas de nom | `conforms = False` |
| `sh:Warning` | Pas d'email | Pas d'impact (avertissement seulement) |
| `sh:Info` | Pas de telephone | Pas d'impact (information seulement) |

> **Bonne pratique** : Utilisez `sh:Warning` pour les proprietes recommandees (best practices) et `sh:Info` pour les suggestions. Reservez `sh:Violation` aux exigences absolues.

---

## 7. Contraintes avancees : operateurs logiques

SHACL permet de combiner des contraintes avec des operateurs logiques :

| Operateur | Propriete SHACL | Semantique |
|-----------|-----------------|------------|
| **OU** | `sh:or` | Au moins une des sous-formes doit etre satisfaite |
| **ET** | `sh:and` | Toutes les sous-formes doivent etre satisfaites |
| **NON** | `sh:not` | La sous-forme ne doit PAS etre satisfaite |
| **OU exclusif** | `sh:xone` | Exactement une sous-forme satisfaite |

### Exemple : un Contact doit avoir un email OU un telephone

In [11]:
from rdflib.collection import Collection

# Forme avancee avec sh:or
adv_shapes = Graph()
adv_shapes.bind("sh", SH)
adv_shapes.bind("ex", EX)
adv_shapes.bind("schema", SCHEMA)
adv_shapes.bind("xsd", XSD)

contact_shape = EX.ContactShape
adv_shapes.add((contact_shape, RDF.type, SH.NodeShape))
adv_shapes.add((contact_shape, SH.targetClass, EX.Contact))

# sh:or - Au moins email OU telephone
# Option A : a un email
option_email = BNode()
email_inner = BNode()
adv_shapes.add((option_email, SH.property, email_inner))
adv_shapes.add((email_inner, SH.path, SCHEMA.email))
adv_shapes.add((email_inner, SH.minCount, Literal(1)))

# Option B : a un telephone
option_phone = BNode()
phone_inner = BNode()
adv_shapes.add((option_phone, SH.property, phone_inner))
adv_shapes.add((phone_inner, SH.path, SCHEMA.telephone))
adv_shapes.add((phone_inner, SH.minCount, Literal(1)))

# Creer la liste RDF pour sh:or
or_list = BNode()
Collection(adv_shapes, or_list, [option_email, option_phone])
adv_shapes.add((contact_shape, SH["or"], or_list))
adv_shapes.add((contact_shape, SH.message, Literal("Un contact doit avoir un email ou un telephone", lang="fr")))

print("Forme ContactShape avec sh:or creee")
print()
print(adv_shapes.serialize(format="turtle"))

Forme ContactShape avec sh:or creee

@prefix ex: <http://example.org/> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix schema1: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:ContactShape a sh:NodeShape ;
    sh:message "Un contact doit avoir un email ou un telephone"@fr ;
    sh:or ( [ sh:property [ sh:minCount 1 ;
                        sh:path schema1:email ] ] [ sh:property [ sh:minCount 1 ;
                        sh:path schema1:telephone ] ] ) ;
    sh:targetClass ex:Contact .




### Test des contraintes avancees

Creons des contacts qui testent differents scenarios : avec email seul, telephone seul, les deux, ou aucun.

In [12]:
# Donnees de test pour les contraintes avancees
contact_data = Graph()
contact_data.bind("ex", EX)
contact_data.bind("schema", SCHEMA)

# Contact 1 : email seulement -> OK
contact_data.add((EX.contact1, RDF.type, EX.Contact))
contact_data.add((EX.contact1, SCHEMA.name, Literal("Alice")))
contact_data.add((EX.contact1, SCHEMA.email, Literal("alice@example.com")))

# Contact 2 : telephone seulement -> OK
contact_data.add((EX.contact2, RDF.type, EX.Contact))
contact_data.add((EX.contact2, SCHEMA.name, Literal("Bob")))
contact_data.add((EX.contact2, SCHEMA.telephone, Literal("+33 1 23 45 67 89")))

# Contact 3 : ni email ni telephone -> VIOLATION (sh:or)
contact_data.add((EX.contact3, RDF.type, EX.Contact))
contact_data.add((EX.contact3, SCHEMA.name, Literal("Charlie")))

# Contact 4 : les deux -> OK
contact_data.add((EX.contact4, RDF.type, EX.Contact))
contact_data.add((EX.contact4, SCHEMA.name, Literal("Diana")))
contact_data.add((EX.contact4, SCHEMA.email, Literal("diana@example.com")))
contact_data.add((EX.contact4, SCHEMA.telephone, Literal("+33 6 12 34 56 78")))

# Valider
conforms3, results3, text3 = validate(
    contact_data,
    shacl_graph=adv_shapes,
    inference='rdfs',
    abort_on_first=False
)

print(f"Conforme : {conforms3}")
print()
print(text3)

Conforme : False

Validation Report
Conforms: False
Results (1):
Constraint Violation in OrConstraintComponent (http://www.w3.org/ns/shacl#OrConstraintComponent):
	Severity: sh:Violation
	Source Shape: ex:ContactShape
	Focus Node: ex:contact3
	Value Node: ex:contact3
	Message: Un contact doit avoir un email ou un telephone



### Interpretation : Contraintes logiques

| Contact | Email | Telephone | Resultat | Raison |
|---------|-------|-----------|----------|--------|
| contact1 | Oui | Non | Conforme | `sh:or` satisfait par email |
| contact2 | Non | Oui | Conforme | `sh:or` satisfait par telephone |
| contact3 | Non | Non | Violation | `sh:or` non satisfait (aucune branche valide) |
| contact4 | Oui | Oui | Conforme | `sh:or` satisfait par les deux (OU inclusif) |

> **Point cle** : `sh:or` est un OU inclusif : avoir les deux options satisfaites est aussi valide. Pour exiger exactement une option, utilisez `sh:xone`.

---

## 8. Exercices

### Exercice 1 : Corriger les erreurs de `person-data.ttl`

Les 5 personnes invalides dans `person-data.ttl` ont chacune une erreur. Corrigez-les par programmation en modifiant le graphe `data_graph`, puis relancez la validation pour verifier que toutes les personnes passent.

**Rappel des erreurs** :
- charlie : pas de nom
- diana : age negatif (-5)
- eve : email mal forme
- frank : nom trop court ("F")
- grace : `knows` pointe vers un non-Person

In [13]:
# Exercice 1 : Corriger les erreurs dans le graphe de donnees

# Recharger les donnees pour partir d'une copie propre
fixed_data = Graph()
fixed_data.parse("data/person-data.ttl", format="turtle")

# TODO : Corrigez chaque erreur
# Correction 1 : Ajouter un nom a charlie
# fixed_data.add((EX.charlie, FOAF.name, Literal("Charlie Durand")))

# Correction 2 : Corriger l'age de diana (remplacer -5 par 25)
# fixed_data.remove((EX.diana, FOAF.age, Literal(-5)))
# fixed_data.add((EX.diana, FOAF.age, Literal(25)))

# Correction 3 : Corriger l'email d'eve
# fixed_data.remove((EX.eve, FOAF.mbox, URIRef("not-an-email")))
# fixed_data.add((EX.eve, FOAF.mbox, URIRef("mailto:eve@example.com")))

# Correction 4 : Allonger le nom de frank
# fixed_data.remove((EX.frank, FOAF.name, Literal("F")))
# fixed_data.add((EX.frank, FOAF.name, Literal("Frank")))

# Correction 5 : Faire de myDog un Person (ou changer le knows)
# fixed_data.remove((EX.myDog, RDF.type, EX.Dog))
# fixed_data.add((EX.myDog, RDF.type, FOAF.Person))

# Valider le graphe corrige
# conforms_fix, _, text_fix = validate(
#     fixed_data, shacl_graph=shapes_graph, inference='rdfs', abort_on_first=False
# )
# print(f"Conforme apres corrections : {conforms_fix}")
# if not conforms_fix:
#     print(text_fix)
# else:
#     print("Toutes les personnes sont maintenant conformes.")

<Graph identifier=Nfdd19e62b709465aad0508c995cf92a0 (<class 'rdflib.graph.Graph'>)>

### Exercice 2 : Forme SHACL pour un Livre

Ecrivez une forme SHACL `ex:BookShape` pour valider des instances de `schema:Book` avec les contraintes suivantes :
- **Titre** (`schema:name`) : obligatoire, chaine, min 1 caractere
- **ISBN** (`schema:isbn`) : optionnel, doit respecter le motif `^(978|979)-\d{1,5}-\d{1,7}-\d{1,6}-\d$`
- **Annee** (`schema:datePublished`) : optionnel, entier entre 1450 (Gutenberg) et 2030
- **Auteur** (`schema:author`) : au moins un, doit etre un `schema:Person`

Puis creez 4 livres (2 valides, 2 invalides) et validez-les.

In [14]:
# Exercice 2 : Completez la forme BookShape

book_shapes = Graph()
book_shapes.bind("sh", SH)
book_shapes.bind("ex", EX)
book_shapes.bind("schema", SCHEMA)
book_shapes.bind("xsd", XSD)

book_shape = EX.BookShape
book_shapes.add((book_shape, RDF.type, SH.NodeShape))
book_shapes.add((book_shape, SH.targetClass, SCHEMA.Book))

# TODO : Ajoutez les 4 contraintes de propriete
# Contrainte 1 : Titre obligatoire (minCount, datatype, minLength)
# ...

# Contrainte 2 : ISBN avec pattern
# ...

# Contrainte 3 : Annee entre 1450 et 2030 (minInclusive, maxInclusive)
# ...

# Contrainte 4 : Au moins un auteur de type schema:Person
# ...

print("Forme BookShape :")
print(book_shapes.serialize(format="turtle"))

# TODO : Creez des donnees de test (2 valides + 2 invalides)
# book_data = Graph()
# ...

# TODO : Validez
# conforms_book, _, text_book = validate(book_data, shacl_graph=book_shapes, ...)
# print(text_book)

Forme BookShape :
@prefix ex: <http://example.org/> .
@prefix schema1: <http://schema.org/> .
@prefix sh: <http://www.w3.org/ns/shacl#> .

ex:BookShape a sh:NodeShape ;
    sh:targetClass schema1:Book .




### Exercice 3 : Contraintes avec severites multiples

Creez une forme SHACL `ex:StudentShape` pour valider des etudiants (`ex:Student`) avec les contraintes suivantes et les severites indiquees :

| Propriete | Contrainte | Severite |
|-----------|-----------|----------|
| `foaf:name` | Obligatoire, chaine | `sh:Violation` |
| `ex:studentId` | Obligatoire, motif `^[A-Z]{2}\d{6}$` | `sh:Violation` |
| `foaf:mbox` | Recommande (minCount 1) | `sh:Warning` |
| `ex:phone` | Souhaitable (minCount 1) | `sh:Info` |

Testez avec 3 etudiants : un complet, un sans email/telephone, un sans nom.

In [15]:
# Exercice 3 : StudentShape avec severites multiples

# TODO : Creez la forme SHACL en Turtle ou par programmation
student_shapes_ttl = """
@prefix sh: <http://www.w3.org/ns/shacl#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .

# TODO : Completez la forme StudentShape
# ex:StudentShape a sh:NodeShape ;
#     sh:targetClass ex:Student ;
#     ...
"""

# TODO : Parsez, creez des donnees de test, et validez
# student_shapes = Graph()
# student_shapes.parse(data=student_shapes_ttl, format="turtle")
# ...

print("Completez l'exercice pour tester les severites SHACL")

Completez l'exercice pour tester les severites SHACL


---

## Resume

### Tableau recapitulatif

| Concept | Ce que nous avons appris |
|---------|-------------------------|
| **SHACL** | Recommandation W3C pour la validation de donnees RDF (monde ferme) |
| **NodeShape** | Forme qui cible des noeuds RDF (par classe ou par URI) |
| **PropertyShape** | Contrainte sur une propriete specifique (cardinalite, type, motif, etc.) |
| **pySHACL** | `validate(data_graph, shacl_graph=..., inference=..., abort_on_first=...)` |
| **Rapport** | Triplet `(conforms, results_graph, results_text)` |
| **Severites** | `sh:Violation` (bloquant), `sh:Warning` (avertissement), `sh:Info` (suggestion) |
| **Operateurs logiques** | `sh:or`, `sh:and`, `sh:not`, `sh:xone` |
| **Construction** | Shapes creees par programmation avec rdflib (BNode, add) |

### Contraintes SHACL -- Reference rapide

| Categorie | Propriete SHACL | Exemple |
|-----------|-----------------|---------|  
| Cardinalite | `sh:minCount`, `sh:maxCount` | `sh:minCount 1` |
| Type | `sh:datatype`, `sh:class`, `sh:nodeKind` | `sh:datatype xsd:string` |
| Chaine | `sh:pattern`, `sh:minLength`, `sh:maxLength` | `sh:pattern "^[A-Z]"` |
| Numerique | `sh:minInclusive`, `sh:maxInclusive` | `sh:minInclusive 0` |
| Valeur | `sh:hasValue`, `sh:in` | `sh:in ("FR" "DE")` |
| Logique | `sh:or`, `sh:and`, `sh:not`, `sh:xone` | Liste de sous-formes |

### Ressources supplementaires

- [W3C SHACL Specification](https://www.w3.org/TR/shacl/) -- Standard complet
- [SHACL Advanced Features](https://www.w3.org/TR/shacl-af/) -- Regles et fonctions
- [pySHACL Documentation](https://github.com/RDFLib/pySHACL) -- Bibliotheque Python
- [SHACL Playground](https://shacl-playground.zazuko.com/) -- Testez vos formes en ligne

---

**Navigation** : [<< 8-PythonRDF](SW-8-PythonRDF.ipynb) | [Index](README.md) | [10-JSONLD >>](SW-10-JSONLD.ipynb)